# Import Libraries and MatplotConfig
Fill in '?' with correct values

In [ ]:
% matplotlib notebook

import ??? as plt
import ??? as pd

#Uncomment the package you need
#import mysql.connector as sql
#import sqlite3 as sql

# Connect to Database
Use the code from your testing scripts to create a connection 'dbconn' for later queries.

Copy paste the info inside previous 'sql.connect()' code to tell python how to connect to the database.

In [ ]:
#dbconn = ???.connect('./classicmodels.db') 
dbconn = ???.connect(user='???', password='???',
                              host='127.0.0.1',
                              database='classicmodels')

# Query the Database with SQL
Use the dbconn connection to run queries against the database, notice dbconn may be reused multiple times.

Mysql users may use MySQL workbench to debug if needed

Answer the questions in comment at top of cell using SQL

In [ ]:
# Select all columns from any five records in customer table
customerFiveRecordsQuery = "SELECT * FROM CUSTOMERS LIMIT 5"

customerFiveRecordsDf = pd.read_sql(customerFiveRecordsQuery, dbconn)

customerFiveRecordsDf

In [ ]:
# Select all columns from any five records joining customers and orders
customerOrdersFiveRecordsQuery = "SELECT * FROM CUSTOMERS \
    INNER JOIN ORDERS \
    ??? CUSTOMERS.customerNumber = ORDERS.customerNumber \
    LIMIT ???" 

customerOrdersFiveRecordsDf = pd.read_sql(customerOrdersFiveRecordsQuery, dbconn)

customerOrdersFiveRecordsDf

In [ ]:
# Select the top five customers by total number of orders
# Hint 'LIMIT' will take the first results of a query, in other words the "top" rows of the result table
# We also introduce alias names using 'AS' to simplify code

customerOrdersTopFiveRecordsQuery = "??? c.customerName AS Name, \
    ???(o.orderNumber) AS Count \
    FROM CUSTOMERS AS c \
    INNER JOIN ORDERS AS o \
    WHERE c.customerNumber ??? o.customerNumber \
    ??? BY c.customerName \
    ORDER BY COUNT(o.orderNumber) DESC \
    LIMIT 5"

customerOrdersTopFiveRecordsDf = pd.read_sql(customerOrdersTopFiveRecordsQuery, dbconn)

customerOrdersTopFiveRecordsDf

# Python Code Operations
Now for some basics in Python and Pandas, we will quickly make the same tables and operation in python, then have a fun code exercise

In [ ]:
customerQuery = "SELECT * FROM CUSTOMERS"
orderQuery = "SELECT * FROM ORDERS"

customerDf = pd.read_sql(customerQuery, dbconn)
orderDf = pd.read_sql(orderQuery, dbconn)

customerOrderDf = customerDf.merge(orderDf, \
                                   left_on = 'customerNumber', \
                                   right_on = 'customerNumber', \
                                   how = 'inner')

customerOrderDf.head(5) #Prints five records same as 'LIMIT 5'

Joining and selecting operations are very similar, even aggregation syntax is close to SQL; however, you will see nested key-value data structures used below in some operations.

The key value structure allows for more control over what we select - for example we are now able to include variables in our column selection, as you will see in a later example

In [ ]:
columns = ['customerName', 'orderNumber']

customerOrderCountDf = customerOrderDf[columns]\
                        .groupby(['customerName'], as_index = False)\
                        .agg({'orderNumber':'count'})\
                        .rename(columns = {'customerName': 'Group','orderNumber': 'Count'})\
                        .sort_values('Count', ascending=False)
                
customerOrderCountDf.head(5)

# Reusable Code
Code becomes powerful when it's reusable, for example what if we were now asked a new problem: "We have heard some complaints from the sales team that some representatives are booking their number under a prexisting representatives contant, can you count the number of sales reps listed for each customer?

Refactor the above code into a function 'colCount' which takes three inputs

Input 0 - The dataframe containing columns for aggregation and counting

Input 1 - The column name which values are used for aggregation

Input 2 - the column name for counting

In [ ]:
def colCount(dataFrameCol, groupColName, countColName):
    columns = [groupColName, countColName]
    columnCountDf = dataFrameCol[columns]\
                        .groupby(columns[0], as_index = False)\
                        .agg({columns[1] : 'count'})\
                        .rename(columns = {columns[0] : 'Group', columns[1]: 'Count'})\
                        .sort_values('Count', ascending=False)
    return columnCountDf

In [ ]:
#Use the function to return the same record count

customerOrderCountFunctionDf = colCount(customerOrderDf, 'customerName', 'orderNumber')

customerOrderCountFunctionDf.head(5)

In [ ]:
#check with other values, such as number of sales reps per customer

salesRepsCustomerCountDf = colCount(customerOrderDf, 'customerName', 'salesRepEmployeeNumber')

salesRepsCustomerCountDf.head(5)

Why did our code return the same values? We did not consider unique values. Since our original join was on customer Id by Order, we got away with a simple row count to determine number of orders...

Because 'count' does not care if the name employee id number appears twice, our second use of the function did not actually answer the question. It also revealed an error in how we thought the code was going to opperate, i.e. a 'bug'

So we now need to fix our function, hint try: 

    .agg({columns[1] : 'nunique'})

In [ ]:
#Customer Order Count should stay the same
customerOrderCountFunctionDf = colCount(customerOrderDf, 'customerName', 'orderNumber')

customerOrderCountFunctionDf.head(5)


In [ ]:
#Sales Rep Count should go down to one each

salesRepsCustomerCountDf = colCount(customerOrderDf, 'customerName', 'salesRepEmployeeNumber')

salesRepsCustomerCountDf.head(5)

# Loops and Conditions
Coding becomes more efficient than manual data entry for not only reproducability and but also looping.

Looping iterates over some set of values doing whatever you tell it to do until you tell it to stop.

So if we wanted to send an email to each of our customers with the name 'Auto' in it (I don't know, maybe it's national automobile day... you can do something like the following:

In [ ]:
import re #built in regular expression library, regex used for parsing text

for name in customerOrderCountFunctionDf.Group:
    if re.match('.*Auto.*', name, re.I): # regex '.*' is any character any number of times and 're.I' is Ignore case
        print "Sending mail to: " + name #Pretend this is a function that sends email

This example combined three extremely useful coding functions: the for loop iterating over our customer name group; an 'if' statement checking each 'name' in the for loop; and a regex match looking for any instance of a word.

FOR each name, IF the name MATCHes the REgex it will PRINT (or do whatever the code tells it to do)

Try it out looking for a series of digits in the customer phone numbers

In [ ]:

for ??? in customerOrderDf.phone:
    if re.match('70.*', ???, re.I): 
        print "Calling: " + value #Pretend this is a function that calls people

# Visualization

Bringing together all our concepts for a final exercise on bar charts

Somebody asked how many unique last names are in each country - it is not our place to ask why, simply answer...

In [ ]:
# Remember our customer order dataframe structure
customerOrderDf.head()

In [ ]:
# Building an aggregated data set of unique last names by country
lastNamesCountry = ???(customerOrderDf, 'country', '???')

lastNamesCountry.head()

In [ ]:
plt.style.???('seaborn-deep')

bar = lastNamesCountry.plot(kind = 'barh', x='???', ???='Count')

bar.legend(??? = "upper right")

# Defining a Class

For information only, nothing to solve

Shows how a new class can create a functional code object

In [ ]:
class countDf:
    
    def __init__(self, df):
        self.df = df
        
    def colCountAll(self, groupColName, countColName):
        columns = [groupColName, countColName]
        columnCountDf = self.df[columns]\
                            .groupby(columns[0], as_index = False)\
                            .agg({columns[1] : 'count'})\
                            .rename(columns = {columns[0] : 'Group', columns[1]: 'Count'})\
                            .sort_values('Count', ascending=False)
        return columnCountDf
    
    def colCountUnique(self, groupColName, countColName):
        columns = [groupColName, countColName]
        columnCountDf = self.df[columns]\
                            .groupby(columns[0], as_index = False)\
                            .agg({columns[1] : 'nunique'})\
                            .rename(columns = {columns[0] : 'Group', columns[1]: 'Count'})\
                            .sort_values('Count', ascending=False)
        return columnCountDf

In [ ]:
countClassDf = countDf(customerOrderDf)

countClassDf.colCountAll('customerName', 'salesRepEmployeeNumber').head(5)

In [ ]:
countClassDf.colCountUnique('customerName', 'salesRepEmployeeNumber').head(5)